In [30]:

import utils.periodogram_dataset_keras as pdk
import utils.contrastive_keras as ck
import utils.data_utils as du
import importlib
import data.spectrogram_dataset as sd
import data.melspectrogram_dataset as md
import data.periodogram_dataset as pd
import data.mfcc_dataset as mfcd
import data.sound as sound
import data.indice.sound_indicies_dataset as si
import data.indice.compute_indice as compute_indice
import data.double_feature_dataset as dfd
import utils.pytorch_impl.cvae as ccc
import utils.pytorch_impl.conv_cvae as conv_ccc
import utils.pytorch_impl.vae as vae
import utils.pytorch_impl.conv_vae as conv_vae
import utils.pytorch_impl.discriminator as discriminator

importlib.reload(vae)
importlib.reload(discriminator)
importlib.reload(conv_vae)
importlib.reload(ccc)
importlib.reload(conv_ccc)
importlib.reload(pd)
importlib.reload(md)
importlib.reload(sound)
importlib.reload(mfcd)
importlib.reload(du)
importlib.reload(si)
importlib.reload(sd)
importlib.reload(dfd)
importlib.reload(compute_indice)




<module 'data.indice.compute_indice' from 'c:\\Users\\tymot\\projects\\001.smartula\\smartula-analysis\\smartula-analysis\\notebooks\\data\\indice\\compute_indice.py'>

In [2]:
import torch

from utils.data_utils import create_valid_sounds_datalist, get_valid_sounds_datalist

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CHECK_FILES = False                             # should we perform sound checking (long)

root_folder = "..\\measurements\\smartulav2"    # root folder with folder containing sounds
valid_filename = "valid-files.txt"              # filename for file which will be contain
                                                # filenames for valid recordings
checked_folders = []
if CHECK_FILES:
    checked_folders = create_valid_sounds_datalist(root_folder, valid_filename, "smrpiclient")
else:
     checked_folders = [os.path.join(root_folder, "smrpiclient0_10082020-19012021"),       
                        os.path.join(root_folder, "smrpiclient3_10082020-19012021"), 
                        os.path.join(root_folder, "smrpiclient5_10082020-19012021"),    
                        os.path.join(root_folder, "smrpiclient6_10082020-19012021"),
                        os.path.join(root_folder, "smrpiclient7_10082020-19012021")]

sound_filenames = get_valid_sounds_datalist(checked_folders, valid_filename)

got 39678 sound filenames read for 5 files/folders


# Variational autoencoder

1. Convolutional version - dedicated to mel spectrograms and spectrograms
2. Simple Autoencoder - fetures extracted from stationary signals and periodograms itself?

In [31]:
import os
import matplotlib.pyplot as plt

from data.periodogram_dataset import PeriodogramDataset
from data.spectrogram_dataset import SpectrogramDataset
from data.melspectrogram_dataset import MelSpectrogramDataset
from data.mfcc_dataset import MfccDataset

from utils.pytorch_impl.vae import VAE, train_vae_model
from utils.data_utils import filter_strlist
from utils.pytorch_impl.conv_vae import ConvolutionalVAE
from utils.pytorch_impl.cvae import cVAE

from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

PATH = 'models\\vae-apprirary-approach-10022021.pth'
TRAIN = True
BATCH_SCALE = True

# model for 
modelVAE = VAE(encoder_layer_sizes=[2048, 512, 64], latent_size=16, decoder_layer_sizes=[64, 512, 2048]).to(device)
# model for mfcc 
mfccVAE = VAE(encoder_layer_sizes=[64, 32], latent_size=4, decoder_layer_sizes=[32, 64]).to(device)
# model for spectrogram
convVAE = ConvolutionalVAE(encoder_conv_sizes=[128, 64, 32, 16], encoder_mlp_sizes=[1024, 512, 128],
                            decoder_conv_sizes=[16, 32, 64, 128], decoder_mlp_sizes=[128, 512, 1024], latent_size=16).to(device)
# model for melspectrogram
melsConvVAE = ConvolutionalVAE(encoder_conv_sizes=[128, 64, 32, 16], encoder_mlp_sizes=[256, 128],
                            decoder_conv_sizes=[16, 32, 64, 128], decoder_mlp_sizes=[128, 256], latent_size=16).to(device)

if TRAIN:
    # training hyperparameters
    batch_size = 16
    epochs = 10
    lr = 1e-3
    weight_decay = 1e-5

    # prepare train data for complete dataset
    train_hives_soundfiles = filter_strlist(sound_filenames, "smrpiclient6", "smrpiclient7")
    print(f"preparing train dataset with len: {len(train_hives_soundfiles)}...")

    # --- HERE CHOOSE CORRECT DATASET AS FEATURE --- #
    # dataset = PeriodogramDataset(train_hives_soundfiles, ["smrpiclient6", "smrpiclient7"], slice_freq=(0, 2048))    
    dataset = SpectrogramDataset(train_hives_soundfiles, ["smrpiclient6", "smrpiclient7"], nfft=4096, hop_len=(4096//3)+30, fmax=2750)
    # dataset = MelSpectrogramDataset(train_hives_soundfiles, ["smrpiclient6", "smrpiclient7"], nfft=4096, hop_len=(4096//3)+30, mels=64)
    # dataset = MfccDataset(train_hives_soundfiles, ["smrpiclient6", "smrpiclient7"], nfft=4096, hop_len=(4096//3)+30, mels=64)

    # --- HERE COOSE CORRECT MODEL --- #
    model = convVAE

    val_amount = int(dataset.__len__() * 0.15)
    train_set, val_set = random_split(dataset, [(dataset.__len__() - val_amount), val_amount])
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=4)

    # train model
    modelVAE = train_vae_model(model, learning_rate=lr, weight_decay=weight_decay, num_epochs=epochs, patience=100, \
                                  dataloader_train=train_loader, dataloader_val=val_loader, scale=BATCH_SCALE, checkpoint_name=PATH)
else:
    if os.path.isfile(PATH) and os.access(PATH, os.R_OK):
        modelVAE.load_state_dict(torch.load(PATH))
        print('model load success!')
    else:
        print(f'ERROR! There is no file: {PATH}')

preparing train dataset with len: 24952...
VAE model training performed on cuda
-> training at epoch 1
  0%|          | 0/1326 [00:08<?, ?it/s]


ValueError: too many values to unpack (expected 2)

### - inference -
here we perform inference to see how our vae models handle data

In [ ]:
import torch
import random 
import matplotlib.pyplot as plt

from data.spectrogram_dataset import SpectrogramDataset
from data.melspectrogram_dataset import MelSpectrogramDataset

inference_hives = filter_strlist(sound_filenames, "smrpiclient6")
idx = random.randrange(len(inference_hives))

spec_dataset = SpectrogramDataset(inference_hives, ["smrpiclient6"], nfft=4096, hop_len=(4096//3)+30, fmax=2750)
melspec_dataset = MelSpectrogramDataset(inference_hives, ["smrpiclient6"], nfft=4096, hop_len=(4096//3)+30, mels=64)

(spectrogram_db, freqs, times), _ = spec_dataset.sample(idx)
melspectrogram_db, _ = melspec_dataset.sample(idx)

with torch.no_grad():
    modelVAE.eval()
    
    fig, axs = plt.subplots(2,2, figsize=(16, 8))

    axs[0][0].set_title('orginal spectrogram')
    axs[0][0].pcolormesh(times, freqs, spectrogram_db.squeeze())
    axs[0][1].set_title('reconstructed spectrogram (convVAE)')
    axs[0][1].pcolormesh(times, freqs, modelVAE(spectrogram_db)[0].cpu().squeeze())
    
    axs[1][0].set_title('original mel-spectrogram')
    axs[1][0].imshow(melspectrogram_db.squeeze())

# Contrastive Variational Autoencoder

-- keras implementation --

In [ ]:
import glob
import random 

from utils.periodogram_dataset_keras import PeriodogramGenerator
from utils.contrastive_keras import contrastive_keras_vae
from utils.data_utils import filter_strlist
from tensorflow import keras

bsize = 64
TRAIN = True
MIXED = True

# background_hives = ["smrpiclient6", "smrpiclient5"]
# target_hives = ["smrpiclient7"]

cvae_keras, _, z_encoder, s_encoder, sb_encoder, cvae_decoder = contrastive_keras_vae(input_dim=2048, intermediate_dim=[512, 64], latent_dim=16,
                                                                                        disentangle=True, gamma=1, batch_size=bsize)

if TRAIN:        
    # prepare train data for complete dataset
    target_soundfilenames = filter_strlist(sound_filenames, "smrpiclient5")
    background_soundfilenames = random.sample(filter_strlist(sound_filenames, "smrpiclient6", "smrpiclient7"), len(target_soundfilenames))
    print(f"preparing target dataset with len: {len(target_soundfilenames)} and background with len: {len(background_soundfilenames)}")
            
    # Prepare dataset and train
    train_generator = PeriodogramGenerator(target_soundfilenames, background_soundfilenames, batch_size=bsize, labels=["smrpiclient5"], slice_freq=(0, 2048))
    history = cvae_keras.fit(train_generator, epochs=10, batch_size=bsize, verbose=1)

    cvae_keras.save_weights('models/cvae_keras_weights_10022021.h5')
    sb_encoder.save_weights('models/sb_encoder_weights_10022021.h5')
    cvae_decoder.save_weights('models/cvae_decoder_weights_10022021.h5')
else:
    cvae_keras.load_weights('models/cvae_keras_weights_10022021.h5')
    sb_encoder.load_weights('models/sb_encoder_weights_10022021.h5')
    cvae_decoder.load_weights('models/cvae_decoder_weights_10022021.h5')
    print(f'model loaded successfully')


-- pytorch implementation --

In [ ]:
import numpy as np

from scipy import signal
from data.sound import read_samples

W = signal.get_window('hanning', 1024, fftbins=False)
samples, sampling_rate = read_samples(target_soundfilenames[0], raw=True)

time_shift = int(1024/2)
times = range(time_shift, len(samples)+1-time_shift, 1024) # centered
frames = [samples[i-time_shift:i+time_shift]*W for i in times] # centered frames

windowed_signal = np.array([frame*W for frame in frames])
print(windowed_signal.shape)


-- PyTorch implementation --

In [20]:
from torchsummary import summary
from utils.pytorch_impl.conv_vae import ConvolutionalVAE
from utils.pytorch_impl.conv_cvae import ConvolutionalCVAE
cvae_model = ConvolutionalCVAE(encoder_conv_sizes=[128, 64, 32, 16], encoder_mlp_sizes=[1024, 512, 128],
                                            decoder_conv_sizes=[16, 32, 64, 128], decoder_mlp_sizes=[128, 512, 1024], latent_size=16)

conv_vae_model = ConvolutionalVAE(encoder_conv_sizes=[128, 64, 32, 16], encoder_mlp_sizes=[1024, 512, 128],
                                            decoder_conv_sizes=[16, 32, 64, 128], decoder_mlp_sizes=[128, 512, 1024], latent_size=16)
summary(conv_vae_model, (4,1,256,64))

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [128, 1, 3, 3], but got 5-dimensional input of size [2, 4, 1, 256, 64] instead

In [14]:
import random

from torch.utils.data import DataLoader, random_split

from utils.pytorch_impl.cvae import cVAE, train_cvae
from utils.pytorch_impl.conv_cvae import ConvolutionalCVAE
from utils.pytorch_impl.discriminator import Discriminator
from utils.data_utils import filter_strlist

from data.spectrogram_dataset import SpectrogramDataset
from data.periodogram_dataset import PeriodogramDataset
from data.double_feature_dataset import DoubleFeatureDataset

import matplotlib.pyplot as plt

target_hives = ["smrpiclient5"]
background_hives = ["smrpiclient6", "smrpiclient7"]

target_soundfilenames = filter_strlist(sound_filenames, *target_hives)
background_soundfilenames = random.sample(filter_strlist(sound_filenames, *background_hives), len(target_soundfilenames))
print(f"preparing target dataset with len: {len(target_soundfilenames)} and background with len: {len(background_soundfilenames)}")

# prepare data for contrastive FC variational autoencoder
# contrastive_dataset = DoubleFeatureDataset(target_soundfilenames, target_hives, background_soundfilenames, background_hives,
#                                                         PeriodogramDataset, scale_db=False, slice_freq=(0, 2048))
# prepare data for contrastive FC variational autoencoder
contrastive_dataset = DoubleFeatureDataset(target_soundfilenames, target_hives, background_soundfilenames, background_hives,
                                                        SpectrogramDataset, nfft=4096, hop_len=(4096//3)+30, fmax=2750)

val_amount = int(contrastive_dataset.__len__() * 0.15)
train_set, val_set = random_split(contrastive_dataset, [(contrastive_dataset.__len__() - val_amount), val_amount])
contrastive_train_dataloader = DataLoader(train_set, batch_size=4, shuffle=True)
contrastive_val_dataloader = DataLoader(val_set, batch_size=4, shuffle=True)
                                
# model = cVAE(encoder_layer_sizes=[2048, 512, 64], latent_size=16, decoder_layer_sizes=[64, 512, 2048])
model = ConvolutionalCVAE(encoder_conv_sizes=[128, 64, 32, 16], encoder_mlp_sizes=[1024, 512, 128],
                                            decoder_conv_sizes=[16, 32, 64, 128], decoder_mlp_sizes=[128, 512, 1024], latent_size=16)
discriminator = Discriminator([64, 128, 16], input_size=2*model.latent_size)

cvae_parameters = { 'learning_rate': 1e-7, 'weight_decay': 1e-3, 'epochs': 10 }
discriminator_parameters  = { 'learning_rate': 0.001, 'weight_decay': 1e-6, 'alpha': 0.01}
cvae = train_cvae(model, cvae_parameters, contrastive_train_dataloader, contrastive_val_dataloader, discriminator, discriminator_parameters)

preparing target dataset with len: 10947 and background with len: 10947
-> training at epoch 0
  0%|          | 0/2327 [00:00<?, ?it/s]torch.Size([4, 1, 256, 64])
torch.Size([4, 1, 128, 128])/torch.Size([4, 1, 256, 64])



RuntimeError: The size of tensor a (128) must match the size of tensor b (64) at non-singleton dimension 3

In [7]:
a = torch.rand(8, 16)
b = torch.ones_like(a)

print(a.shape)
print(b.shape)

torch.cat((a, b), axis=1).shape

torch.Size([8, 16])
torch.Size([8, 16])


torch.Size([8, 32])

-> Here we test generalization ability by testing MSE loss for unseen hive. We start with VAE and then contrastive Autoencoder

In [ ]:
import random
import torch
import numpy as np
import torch.nn.functional as F

from tqdm import tqdm
from utils.periodogram_dataset import PeriodogramDataset
from utils.data_utils import filter_strlist

hut_filenames = filter_strlist(sound_filenames, "smrpiclient3")
hut_dataset = PeriodogramDataset(hut_filenames, ["smrpiclient3"], slice_freq=(0, 2048))

test_loss = []
with torch.no_grad():
    modelVAE.eval()
    for input_data in tqdm(hut_dataset):
        if input_data:
            input_data = torch.Tensor(input_data[0]).to(device)
            output, mean, var = modelVAE(input_data.to(device))
            test_loss.append(F.mse_loss(output, input_data).cpu().numpy())

test_loss_avg = np.mean(test_loss)

print(f'\r\nVAE test loss: {test_loss_avg:.8f}')
                                    

In [ ]:
import tensorflow as tf

from operator import itemgetter 
from utils.periodogram_dataset_keras import PeriodogramGenerator, read_sound_file
from tqdm import tqdm 

hut_sounds =  np.array([read_sound_file(f, (0, 2048)) for f in tqdm(hut_filenames)])
mean, var, latent = sb_encoder.predict(hut_sounds)
zeros = np.zeros_like(latent)
decoder_input = np.concatenate((zeros, latent), axis=1)
cvae_output = cvae_decoder.predict(decoder_input)
loss = tf.keras.losses.mean_squared_error(hut_sounds, cvae_output)

print(f'cVAE test loss:: {np.mean(loss)}')

In [ ]:
subset_indices = np.random.randint(0, len(background_soundfilenames), len(hut_sounds))

background_train_sound_files = list(itemgetter(*subset_indices)(background_soundfilenames))
bg_sounds = np.array([read_sound_file(f, (0, 2048)) for f in tqdm(background_train_sound_files)])
mean, var, latent = sb_encoder.predict(bg_sounds)
zeros = np.zeros_like(latent)
decoder_input = np.concatenate((zeros, latent), axis=1)
cvae_output = cvae_decoder.predict(decoder_input)
loss = tf.keras.losses.mean_squared_error(bg_sounds, cvae_output)

print(f'MSE loss for background test: {np.mean(loss)}')


# OLD ANALYSIS BELOW
##  |
##  |
## \/
# Data Preprocessing

Here we load train data sound samples and prepare spectrogram, periodogram and mfcc features (along with some data to visualize this). We should provide data with **utc timestamps** as it will be shifted with `timezone_offset_hours` var. What we also do is remove those samples which has strange rms signal. Threshold 0.8 was chosen based on `plot_distribution` output.


In [ ]:
import numpy as np
import glob
import librosa

from tqdm import tqdm
from datetime import datetime, timedelta
from scipy.io import wavfile
from scipy.fftpack import fft, fftfreq

sound_time_ms = 2000
# ~93 ms for fft window
nfft = 4096
# ~34% overlapping
hop_len = (nfft//3) + 30
# This can be manipulated to adjust number of bins for conv layer
fmax = 2750

hives_data = []
rmses = {}
max_to_norm = 0

if DATA_INIT:
    for idx, hive_id in enumerate(hives_ids):
        sound_files = [f for f in glob.glob(f"..\\measurements\\smartulav2\\{hive_id}_*\\*.wav")]
        print(f"Sound data preparation for hive: {hive_id} which has {len(sound_files)} recordings...", end=' ', flush=True)
        for file in tqdm(sound_files):
            sample_rate, sound_samples = wavfile.read(file)
            sound_samples = sound_samples.T[0]/(2.0**31)
            rms = np.sqrt(sum(sound_samples**2)/len(sound_samples))
            if rms < 0.7:    # that threshold was observed from plot_distribution() function
                # calculate timestamp
                filename = file.rsplit('\\', 1)[-1]
                utc_timestamp = filename[filename.index('-')+1:].rsplit(".wav")[0]
                sound_datetime = datetime.strptime(utc_timestamp, '%Y-%m-%dT%H-%M-%S') + timedelta(hours=timezone_offset_hours)
                
                # calculate mfcc feature
                mfccs = librosa.feature.mfcc(y=sound_samples, sr=sample_rate, n_fft=nfft, hop_length=hop_len, n_mfcc=13)
                np_mfcc_avg = np.mean(mfccs, axis=1)
                
                # calculate spectrogram
                spectrogram = librosa.core.stft(sound_samples, n_fft=nfft, hop_length=hop_len)
                spectrogram_magnitude = np.abs(spectrogram)
                spectrogram_phase = np.angle(spectrogram)
                spectrogram_db = librosa.amplitude_to_db(spectrogram_magnitude, ref=np.max)
                frequencies = librosa.fft_frequencies(sr=sample_rate, n_fft=nfft)
                times = (np.arange(0, spectrogram_magnitude.shape[1])*hop_len)/sample_rate
                freq_slice = np.where((frequencies < fmax))
                frequencies = frequencies[freq_slice]
                spectrogram_db = spectrogram_db[freq_slice, :][0]
                spectrogram_mean = np.mean(spectrogram_db, axis=1)
                # decimate?
                # spectrogram_db_decimated = decimate(spectrogram_db.T, 4).T
                # frequencies_decimated = decimate(frequencies, 4)

                #calculate periodogram
                periodogram = fft(sound_samples, n=sample_rate)
                periodogram = abs(periodogram[1:int(len(periodogram)/2)])
                periodogram_freq = fftfreq(len(sound_samples)//(sound_time_ms//1000), 1/sample_rate)
                periodogram_freq = periodogram_freq[:(len(periodogram_freq)//2)-1]
                
                hives_data.append(
                    {
                        'datetime': sound_datetime,
                        'id': hive_id,
                        'samples': sound_samples,
                        'freq':
                            {
                                'spectrogram':
                                    {
                                        'frequencies': frequencies,
                                        'time': times,
                                        'spectrogram_full_db': spectrogram_db,
                                        'spectrogram_mean': spectrogram_mean
                                    },
                                'periodogram':
                                    {
                                        'frequencies': periodogram_freq,
                                        'samples': periodogram
                                    }
                            },
                        'features':
                            {
                                'mfcc_avg': np_mfcc_avg
                            }
                    }
                )
        print(" done.")
        
    print("saving data on disc...", end=' ')
    np.save('data/raw_hives_data.npy', hives_data, allow_pickle=True)
    print("done.")
else:
    hives_data = np.load('data/raw_hives_data.npy', allow_pickle=True)
    
print(f"got full dataset of {len(hives_data)} sound samples")

### Data preprocessing 

Here we perform scaling standarization etc.
 

In [ ]:
import pandas as pd

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tqdm import tqdm

sc = StandardScaler()
mm = MinMaxScaler()

# extract every spectrogram from hives data list and standarize every periodogram from stft 
# so data for spectrogram will have zero mean and unit variance + scale every standarized periodogram
print('preparing spectrograms...', end=' ', flush=True)
spectrograms = [hive_data['freq']['spectrogram']['spectrogram_full_db'] for hive_data in hives_data]
# standarized_spectrograms = [sc.fit_transform(spectrogram.T).T for spectrogram in tqdm(spectrograms)]
scaled_spectrogram = [mm.fit_transform(spectrogram.T).T for spectrogram in tqdm(spectrograms, position=0, leave=True)]
print('done.')

# get datatime, names and mfcc
print('getting mfccs', end=' ')
datetimes = [hive_data['datetime'] for hive_data in hives_data]
names = [hive_data['id'] for hive_data in hives_data]
mfccs = [hive_data['features']['mfcc_avg'] for hive_data in hives_data]
print('done.')

# standarize and scale mean spectrogram for sounds
print('preparing spectrogram means', end=' ')
spectrograms_mean = [hive_data['freq']['spectrogram']['spectrogram_mean'] for hive_data in hives_data]
standarized_spectrograms_mean = StandardScaler().fit_transform(spectrograms_mean)
scaled_spectrograms_means = MinMaxScaler().fit_transform(standarized_spectrograms_mean)
print('done.')

# prepare truncated periodogram
end_frequency = 2048
print(f'preparing truncated periodograms ({end_frequency})', end=' ', flush=True)
periodograms = [hive_data['freq']['periodogram']['samples'][:end_frequency] for hive_data in hives_data]
periodograms = [mm.fit_transform(perio.reshape(-1, 1)).T for perio in tqdm(periodograms, position=0, leave=True)]
print('done.')

sounds = list(zip(scaled_spectrogram, mfccs, scaled_spectrograms_means, periodograms, datetimes, names))

sounds_data = pd.DataFrame(sounds, columns=['spectrogram', 'mfccs', 'spectrogram_mean', 'periodogram', 'datetime', 'name'])
sounds_data['datetime'] = pd.to_datetime(sounds_data['datetime'])
sounds_hive_data = sounds_data[sounds_data['name'] == hive_under_analysis]

print(f"Got dataset of size: {len(sounds)}")

## Train CONTRASTIVE AE - keras

In [ ]:
sounds_data[sounds_data['name'] == 'smrpiclient7']['periodogram'].to_numpy().shape

In [ ]:
from utils.contrastive_keras import contrastive_keras_vae
from sklearn.preprocessing import MinMaxScaler

target_keras = np.stack(sounds_data[sounds_data['name'] == 'smrpiclient7']['periodogram'].to_numpy()[:3000]).squeeze()
background_keras = np.stack(sounds_data[sounds_data['name'] == 'smrpiclient6']['periodogram'].to_numpy()[:3000]).squeeze()

print(f'training cvae (keras) with target of shape: {target_keras.shape} '
    f'and background of shape: {background_keras.shape}')

assert(target_keras.shape == background_keras.shape)

cvae_keras, cvae_fg, z_encoder_keras, s_encoder_keras, cvae_keras_decoder = contrastive_keras_vae(
    input_dim=2048, intermediate_dim=512, latent_dim=16, disentangle=True, gamma=1)
history = cvae_keras.fit([target_keras, background_keras], epochs=50, batch_size=100, 
                         validation_data=([target_keras, background_keras], None), verbose=1)

## Train BASIC AE
Here we train basic fully connected autoencoder on data from particular hive

In [ ]:
import torch 
import os

from torch.utils import data as tdata
from utils.data_utils import prepare_dataset1d
from utils.autoencoder_utils import BasicAutoencoder, train_model

TRAIN_MODEL = False
PATH = 'basic_ae.pth'

modelBasicAE = BasicAutoencoder().to(device)

if TRAIN_MODEL:
    train_dataloader, val_dataloader = prepare_dataset1d(sounds_hive_data['spectrogram_mean'], train_ratio=0.8, batch_size=64)
    modelBasicAE = train_model(modelBasicAE,
                           learning_rate=1e-3, weight_decay=1e-5, num_epochs=10, patience=20,
                           dataloader_train=train_dataloader, dataloader_val=val_dataloader,
                           checkpoint_name=PATH)
else:
    if os.path.isfile(PATH) and os.access(PATH, os.R_OK):
        modelBasicAE.load_state_dict(torch.load(PATH))
        print('model load success!')
    else:
        print(f'ERROR! There is no file: {PATH}')

## Train CONV AE

Here we train convolutional autoencoder on data from particular hive

In [ ]:
from torch.utils import data as tdata
from utils.data_utils import prepare_dataset2d
from utils.autoencoder_utils import ConvAutoencoder, train_model

TRAIN_MODEL = True
PATH = 'conv_ae.pth'

modelConvAE = ConvAutoencoder().to(device)

if TRAIN_MODEL:
    train_set, val_set = prepare_dataset2d(sounds_hive_data['spectrogram'], train_ratio=0.8)
    
    dataloader_train = tdata.DataLoader(train_set, batch_size=6, shuffle=True)
    dataloader_val = tdata.DataLoader(val_set, batch_size=6, shuffle=True)
    
    modelConvAE = train_model(modelConvAE,
                               learning_rate=1e-3, weight_decay=1e-6, num_epochs=100, patience=20,
                               dataloader_train=dataloader_train, dataloader_val=dataloader_val,
                               checkpoint_name=PATH)
else:
    if os.path.isfile(PATH) and os.access(PATH, os.R_OK):
        modelConvAE.load_state_dict(torch.load(PATH))
        print('model load success!')
    else:
        print(f'ERROR! There is no such file: {PATH}')

## Train VAE

Here we train vae autoencoder on data from particular hive

In [ ]:
import torch

from torch.utils import data as tdata
from torchvision import transforms
from utils.data_utils import prepare_dataset1d
from utils.contrastive_pytorch import VAE, train_vae_model
from utils.customdataset import CustomDataset

TRAIN_MODEL = True
PATH = 'vae.pth'

modelVAE = VAE(encoder_layer_sizes=[2048, 512, 64], latent_size=16, decoder_layer_sizes=[64, 512, 2048])

if TRAIN_MODEL:
    data = torch.Tensor(sounds_hive_data['periodogram'])
    data = data[:, None, :]
    idx_split = data.size(0)*80//100
    train_dataset = CustomDataset(data[:idx_split], should_scale=True)
    val_dataset = CustomDataset(data[idx_split:], should_scale=True)
    train_dataloader = tdata.DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_dataloader = tdata.DataLoader(val_dataset, batch_size=64, shuffle=True)
    
    modelVAE = train_vae_model(modelVAE, learning_rate=1e-3, weight_decay=1e-5, num_epochs=100, patience=100,
                              dataloader_train=train_dataloader, dataloader_val=val_dataloader, checkpoint_name=PATH)
else:
    if os.path.isfile(PATH) and os.access(PATH, os.R_OK):
        modelVAE.load_state_dict(torch.load(PATH))
        print('model load success!')
    else:
        print(f'ERROR! There is no such file: {PATH}')

# Encode data

In [ ]:
import matplotlib.pyplot as plt
import random
import torch
from scipy.io import wavfile

# read just sample file as probably all has the same sampling rate
sample_rate, samples = wavfile.read('C:\\Users\\tymot\\projects\\001.smartula\\smartula-analysis\\smartula-analysis'
                           '\\measurements\\smartulav2\\smrpiclient6_10082020-15092020\\DEADBEEF94-2020-08-10T03-11-50.wav')
idx = random.randint(0, sounds_hive_data.shape[0] - 1)
idx = 2240
with torch.no_grad():   
    modelConvAE.eval()
    modelBasicAE.eval()
    modelVAE.eval()
    
    # Get sound from sounds hive data (with respect to hive under analysis)
    sound = sounds_hive_data.iloc[idx]
    
    fig, axs = plt.subplots(2, 2, figsize=(12,8))
    fig.suptitle(f'Convolutional Autoencoder real vs output for sample id: {idx}')
    fig.tight_layout(pad = 3.0)

    frequencies = librosa.fft_frequencies(sr=sample_rate, n_fft=nfft)
    freq_slice = np.where((frequencies < fmax))
    frequencies = frequencies[freq_slice]
    times = (np.arange(0, sound['spectrogram'].shape[1])*hop_len)/sample_rate
    elem = sound['spectrogram']
    elem = elem[None, None, :, :]
    elem = torch.Tensor(elem).to(device)
    
    elem_mean = sound['spectrogram_mean']
    elem_mean = elem_mean[None, :]
    elem_mean = torch.Tensor(elem_mean).to(device)
    
    axs[0][0].set_title('real spectrogram')
    axs[0][0].pcolormesh(times, frequencies, sound['spectrogram'])
    axs[0][1].set_title('encoded conv2d')
    axs[0][1].pcolormesh(times, frequencies, modelConvAE(elem.to(device)).cpu().numpy().squeeze())
    
    # prepare data for vae prediction
    freqs = np.arange(sound['periodogram'].shape[1])
    input_data = torch.Tensor(sound['periodogram']).to(device)
    vae_output, _, _ = modelVAE(input_data)
    
    # prepare data for cvae prediction
    s_mean, s_log_var, s = s_encoder_keras.predict(sound['periodogram'])
    latent = np.concatenate((np.zeros_like(s), s), axis = 1)
    cvae_output = cvae_keras_decoder.predict(latent)

    axs[1][0].set_title('spectrogram mean')
    axs[1][0].plot(sound['spectrogram_mean'], label='real', color='blue')
    axs[1][0].plot(modelBasicAE(elem_mean).cpu().numpy().squeeze(), label='encoded', color='red')
    axs[1][0].legend()
    
    axs[1][1].set_title('periodogram')
    axs[1][1].plot(freqs, periodogram_df, label='real', color='blue')
    axs[1][1].plot(freqs, vae_output.cpu().numpy().squeeze(), label='vae encoded', color='red')
    axs[1][1].plot(freqs, cvae_output.squeeze(), label='cvae encoded', color='green')
    axs[1][1].legend()
    
    plt.show()

# Add temperature/humidity/gas

In [ ]:
from utils.data_utils import read_sensor_data

start_time = '2020-08-10 00:00:00'
end_time = '2020-09-16 00:00:00'
print(f"extracting data for hive under analysis: {hive_under_analysis} from {start_time} to {end_time}...")

df_hives_sound = pd.DataFrame(sounds_data)
df_hive_sound_ua = df_hives_sound[(df_hives_sound['name'] == hive_under_analysis)
                                 & (df_hives_sound['datetime'] > start_time)
                                 & (df_hives_sound['datetime'] < end_time)]
df_hive_sound_ua.set_index('datetime', inplace=True)
print(f"-> prepared base of {df_hive_sound_ua.count()['spectrogram']} number of sound spectrums <-")

df_hive_temperature_ua = read_sensor_data('..//measurements//smartulav2//sulmin-10082020-15092020-inside-temperature.csv',
                                          hive_under_analysis, hives_ids, start_time, end_time, timezone_offset_hours, 'temperature')
df_hive_humidity_ua = read_sensor_data('..//measurements//smartulav2//sulmin-10082020-15092020-inside-humidity.csv',
                                       hive_under_analysis, hives_ids, start_time, end_time, timezone_offset_hours, 'humidity')
df_hive_alcohol_ua = read_sensor_data('..//measurements//smartulav2//sulmin-10082020-15092020-inside-alcohol.csv',
                                      hive_under_analysis, hives_ids, start_time, end_time, timezone_offset_hours, 'alcohol')
df_hive_aceton_ua = read_sensor_data('..//measurements//smartulav2//sulmin-10082020-15092020-inside-aceton.csv',
                                     hive_under_analysis, hives_ids, start_time, end_time, timezone_offset_hours, 'aceton')
df_hive_amon_ua = read_sensor_data('..//measurements//smartulav2//sulmin-10082020-15092020-inside-jon-amonowy.csv',
                                   hive_under_analysis, hives_ids, start_time, end_time, timezone_offset_hours, 'jon-amonowy')
df_hive_toluen_ua = read_sensor_data('..//measurements//smartulav2//sulmin-10082020-15092020-inside-toluen.csv',
                                     hive_under_analysis, hives_ids, start_time, end_time, timezone_offset_hours, 'toluen')
df_hive_co2_ua = read_sensor_data('..//measurements//smartulav2//sulmin-10082020-15092020-inside-co2.csv',
                                    hive_under_analysis, hives_ids, start_time, end_time, timezone_offset_hours, 'co2')
df_hive_siarkowodor_ua = read_sensor_data('..//measurements//smartulav2//sulmin-10082020-15092020-inside-siarkowodor.csv',
                                          hive_under_analysis, hives_ids, start_time, end_time, timezone_offset_hours, 'siarkowodor')
df_hive_metanotiol_ua = read_sensor_data('..//measurements//smartulav2//sulmin-10082020-15092020-inside-metanotiol.csv',
                                         hive_under_analysis, hives_ids, start_time, end_time, timezone_offset_hours, 'metanotiol')
df_hive_trimetyloamina_ua = read_sensor_data('..//measurements//smartulav2//sulmin-10082020-15092020-inside-trimetyloamina.csv',
                                         hive_under_analysis, hives_ids, start_time, end_time, timezone_offset_hours, 'trimetyloamina')
df_hive_wodor_ua = read_sensor_data('..//measurements//smartulav2//sulmin-10082020-15092020-inside-wodor.csv',
                                    hive_under_analysis, hives_ids, start_time, end_time, timezone_offset_hours, 'wodor')
df_hive_co_ua = read_sensor_data('..//measurements//smartulav2//sulmin-10082020-15092020-inside-co.csv',
                                 hive_under_analysis, hives_ids, start_time, end_time, timezone_offset_hours, 'co')


Check autocorrelation for specific features

In [ ]:
from utils.data_utils import merge_dataframes_ontimestamp, merge_columns
from utils.autoencoder_utils import conv2d_encode, basic_ae_encode
from utils.contrastive_pytorch import vae_encode
from utils.contrastive_keras import cvae_encode

df_hive_data = merge_dataframes_ontimestamp(df_hive_sound_ua,
                                            df_hive_temperature_ua, df_hive_humidity_ua,
                                            df_hive_alcohol_ua, df_hive_aceton_ua, df_hive_amon_ua, df_hive_toluen_ua, df_hive_co2_ua,
                                            df_hive_siarkowodor_ua, df_hive_metanotiol_ua, df_hive_trimetyloamina_ua, df_hive_wodor_ua,
                                            df_hive_co_ua)

df_hive_data['conv_ae'] = conv2d_encode(modelConvAE, df_hive_data['spectrogram'].to_list())
df_hive_data['basic_ae'] = basic_ae_encode(modelBasicAE, df_hive_data['spectrogram_mean'].to_list())
df_hive_data['vae'] = vae_encode(modelVAE, df_hive_data['periodogram'].to_list()).tolist()
df_hive_data['cvae_s'] = cvae_encode(s_encoder_keras, df_hive_data['periodogram']).tolist()
df_hive_data['cvae_z'] = cvae_encode(z_encoder_keras, df_hive_data['periodogram']).tolist()

df_hive_data['bae_feature_vector'] = merge_columns(df_hive_data, ['basic_ae', 'humidity', 'temperature', 
                                                                  'alcohol', 'aceton', 'jon-amonowy', 'toluen', 'co2', 'trimetyloamina', 'co'])
df_hive_data['conv_feature_vector'] = merge_columns(df_hive_data, ['conv_ae', 'humidity', 'temperature', 
                                                                  'alcohol', 'aceton', 'jon-amonowy', 'toluen', 'co2', 'trimetyloamina', 'co'])
df_hive_data['mfcc_feature_vector'] = merge_columns(df_hive_data, ['mfccs', 'humidity', 'temperature', 
                                                                  'alcohol', 'aceton', 'jon-amonowy', 'toluen', 'co2', 'trimetyloamina', 'co'])
df_hive_data['vae_feature_vector'] = merge_columns(df_hive_data, ['vae', 'humidity', 'temperature', 
                                                                  'alcohol', 'aceton', 'jon-amonowy', 'toluen', 'co2', 'trimetyloamina', 'co'])
df_hive_data['cvae_s_feature_vector'] = merge_columns(df_hive_data, ['cvae_s', 'humidity', 'temperature', 
                                                                  'alcohol', 'aceton', 'jon-amonowy', 'toluen', 'co2', 'trimetyloamina', 'co'])
df_hive_data['cvae_z_feature_vector'] = merge_columns(df_hive_data, ['cvae_z', 'humidity', 'temperature', 
                                                                  'alcohol', 'aceton', 'jon-amonowy', 'toluen', 'co2', 'trimetyloamina', 'co'])

# SVM classification 

In [ ]:
from sklearn.preprocessing import StandardScaler
from utils.data_utils import search_best_night_day

start_hours = [20, 21, 22, 23, 0, 1, 2, 3, 4]

df_hive_data_scaled = pd.DataFrame(df_hive_data)

# data for convolutional autoencoder
df_hive_data_scaled['conv_feature_vector'] = StandardScaler().fit_transform(df_hive_data['conv_feature_vector'].values.tolist()).tolist()

# data for basic autoencoder
df_hive_data_scaled['bae_feature_vector'] = StandardScaler().fit_transform(df_hive_data['bae_feature_vector'].values.tolist()).tolist()

# data for mfcc features
df_hive_data_scaled['mfcc_feature_vector'] = StandardScaler().fit_transform(df_hive_data['mfcc_feature_vector'].values.tolist()).tolist()

# data for vae features
df_hive_data_scaled['vae_feature_vector'] = StandardScaler().fit_transform(df_hive_data['vae_feature_vector'].values.tolist()).tolist()

# data for cvae s features
df_hive_data_scaled['cvae_s_feature_vector'] = StandardScaler().fit_transform(df_hive_data['cvae_s_feature_vector'].values.tolist()).tolist()

# data for cvae s features
df_hive_data_scaled['cvae_z_feature_vector'] = StandardScaler().fit_transform(df_hive_data['cvae_z_feature_vector'].values.tolist()).tolist()

# data for plain mfcc 
mfccs = [hive_data['features']['mfcc_avg'] for hive_data in hives_data if hive_data['id'] == hive_under_analysis]
mfccs = StandardScaler().fit_transform(mfccs)
datetimes = [hive_data['datetime'] for hive_data in hives_data if hive_data['id'] == hive_under_analysis]
mfccs_data = list(zip(datetimes, mfccs))
pd_mfcc_data = pd.DataFrame(mfccs_data, columns=['datetime', 'mfcc'])
pd_mfcc_data.set_index('datetime', inplace=True)

# calculate one class SVM match
print('calculating mfccs match...', end=' ', flush=True)
mfcc_accs = search_best_night_day(pd_mfcc_data, 'mfcc', days_as_test=10, start_hours=start_hours, max_shift=6, verbose=0)
print(f'done. {len(mfcc_accs)}/{len(mfcc_accs[0])}')
print('calculating conv ae feature vector match...', end=' ', flush=True)
conv_ae_accs = search_best_night_day(df_hive_data_scaled, 'conv_feature_vector', days_as_test=10, start_hours=start_hours, max_shift=6, verbose=0)
print(f'done. {len(conv_ae_accs)}/{len(conv_ae_accs[0])}')
print('calculating basic ae feature vector match...', end=' ', flush=True)
bae_accs = search_best_night_day(df_hive_data_scaled, 'bae_feature_vector', days_as_test=10, start_hours=start_hours, max_shift=6, verbose=0)
print(f'done. {len(bae_accs)}/{len(bae_accs[0])}')
print('calculating mfccs extended feature vector match...', end=' ', flush=True)
mffce_accs = search_best_night_day(df_hive_data_scaled, 'mfcc_feature_vector', days_as_test=10, start_hours=start_hours, max_shift=6, verbose=0)
print(f'done. {len(mffce_accs)}/{len(mffce_accs[0])}')
print('calculating vae feature vector match...', end=' ', flush=True)
vae_accs = search_best_night_day(df_hive_data_scaled, 'vae_feature_vector', days_as_test=10, start_hours=start_hours, max_shift=6, verbose=0)
print(f'done. {len(vae_accs)}/{len(vae_accs[0])}')
print('calculating cvae s feature vector match...', end=' ', flush=True)
cvae_accs_s = search_best_night_day(df_hive_data_scaled, 'cvae_s_feature_vector', days_as_test=10, start_hours=start_hours, max_shift=6, verbose=0)
print(f'done. {len(cvae_accs_s)}/{len(cvae_accs_s[0])}')
print('calculating cvae z feature vector match...', end=' ', flush=True)
cvae_accs_z = search_best_night_day(df_hive_data_scaled, 'cvae_z_feature_vector', days_as_test=10, start_hours=start_hours, max_shift=6, verbose=0)
print(f'done. {len(cvae_accs_z)}/{len(cvae_accs_z[0])}')

In [ ]:
from utils.data_utils import plot_hour_shift

plot_hour_shift(mfcc_accs, conv_ae_accs, bae_accs, mffce_accs, vae_accs, cvae_accs_s, cvae_accs_z,
                labels_list=['mfcc', 'conv', 'bae', 'mfcce', 'vae', 'cvae_s', 'cvae_z'],
                xticklabels=[str(start_hour) for start_hour in start_hours],
                save_path = 'data\\outputs\\zs_encoder_0_s.png')

Visualize on 2D map, we basically perform TSNE and PCA dimension reduction in order to visualize night and day. Probably this will be not efficent but it is worth to give a shot.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

start_hour = 23
end_hour = 2

reduce_df = pd.DataFrame(df_hive_data)
reduce_df['feature_vector'] = StandardScaler().fit_transform(df_hive_data['bae_feature_vector'].values.tolist()).tolist()

reduced_ae_pca = PCA(n_components=2).fit_transform(reduce_df['feature_vector'].values.tolist())
reduced_ae_tsne =  TSNE(n_components=2, perplexity=100, learning_rate=500).fit_transform(reduce_df['feature_vector'].values.tolist())
is_night_list = (reduce_df.index.hour >= start_hour) | (reduce_df.index.hour <= end_hour)
                
colors = ['red', 'green', 'blue', 'yellow']
labels = ['day', 'night']

fig, axs = plt.subplots(2, figsize=(10,10))

axs[0].scatter(x=[data[0] for data in reduced_ae_pca],
               y=[data[1] for data in reduced_ae_pca],
               c=[colors[night] for night in is_night_list],
              alpha=0.3)
axs[0].set_title('PCA')

axs[1].scatter(x=[data[0] for data in reduced_ae_tsne],
               y=[data[1] for data in reduced_ae_tsne],
               c=[colors[night] for night in is_night_list],
              alpha=0.3)
axs[1].set_title('TSNE')

plt.show()

In [ ]:
pca = PCA().fit(reduce_df['feature_vector'].values.tolist())

plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');




In [ ]:
import matplotlib

def plot_distribution(distribution_dict, bin_size):
    """ Plotting distribiution for dictionary elements"""
    colors = ['blue', 'green', 'red', 'yellow', 'black', 'pink', 'purple']
    rms_max = 0
    rms_min = 65535
    for k, v in rmses.items():
        if np.max(v) > rms_max:
            rms_max = np.max(v)
        if np.min(v) < rms_min:
            rms_min = np.min(v)
        
    plt.figure()
    for idx, (k, v) in enumerate(distribution_dict.items()):
        plt.hist(v, color=colors[idx%len(colors)], bins=int(np.abs(rms_max-rms_min)/bin_size))
    plt.show()

Part of code for calculating autocorrelaction for specific feature

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf

%matplotlib widget

features = ['conv_ae', 'humidity', 'temperature',
            'alcohol', 'aceton', 'jon-amonowy',
            'toluen', 'co2', 'siarkowodor',
            'metanotiol', 'trimetyloamina', 'wodor', 'co']

feature = features[12]
data_to_autocorr = df_hive_co_ua

roll_len = 3
interval = (data_to_autocorr.index[2] - data_to_autocorr.index[1]).seconds//60%60

y2 = data_to_autocorr[feature].rolling(window=roll_len).mean().values
y_corr = y2[roll_len:]
x_corelation = np.arange(start=0, step=2, stop=150)

fig, axes = plt.subplots(1, figsize=(8,5))
x = plot_acf(y_corr, lags=x_corelation, zero=False, ax=axes)
x_raw = acf(y_corr, nlags=150)
axes.set_title(f'{feature} autocorrelaction')
axes.set_xlabel(f'Lag (1 lag = {interval} minutes)') 
axes.set_ylabel('Correlation')
axes.set_xticks(np.arange(0, 151, step=10))

print(f'{feature} with max {max(x_raw[60:]):.2f} at {60 + np.argmax(x_raw[60:])}')

# temperature with max 0.74 at 93 (15 mint)
# humidity with max 0.58 at 92 (15 min)
# alcohol with max 0.53 at 134 (10 min)
# aceton with max 0.52 at 133 (10 min)
# jon-amonowy with max 0.57 at 133 (10 min)
# toluen with max 0.52 at 134 (10 min)
# co2 with max 0.54 at 133 (10 min)
# siarkowodor with max 0.16 at 142 (10 min)
# metanotiol with max 0.34 at 140 (10 min)
# trimetyloamina with max 0.56 at 138 (10 min)
# wodor with max 0.14 at 142 (10 min)
# co with max 0.62 at 134 (10 min)

In [ ]:
import importlib
import utils.autoencoder_utils as ae
import utils.data_utils as du
import utils.customdataset as cd
import utils.contrastive_pytorch as cp
import utils.contrastive_keras as ck
import utils.periodogram_dataset as pd

importlib.reload(pd)
